In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df=pd.read_csv("train.csv")

In [5]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
df["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
df.shape

(7613, 5)

In [8]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5080 entries, 31 to 7581
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5080 non-null   int64 
 1   keyword   5080 non-null   object
 2   location  5080 non-null   object
 3   text      5080 non-null   object
 4   target    5080 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 238.1+ KB


In [13]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [14]:
df["clean_text"]=df["text"].apply(cleantext)

In [15]:
x = df["clean_text"]
y = df["target"]

In [16]:
x.info()

<class 'pandas.core.series.Series'>
Int64Index: 5080 entries, 31 to 7581
Series name: clean_text
Non-Null Count  Dtype 
--------------  ----- 
5080 non-null   object
dtypes: object(1)
memory usage: 79.4+ KB


In [17]:
y.info()

<class 'pandas.core.series.Series'>
Int64Index: 5080 entries, 31 to 7581
Series name: target
Non-Null Count  Dtype
--------------  -----
5080 non-null   int64
dtypes: int64(1)
memory usage: 79.4 KB


In [18]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)


In [19]:
sentlen = []

for sent in df["clean_text"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen
df.head()

,id,keyword,location,text,target,clean_text,SentLen
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze http,5
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt http,7
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking news nigeria flag set abl...,9
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,cry set ablaze,3
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze http,8


In [20]:
max(sentlen)

23

In [21]:
np.quantile(sentlen, 0.95)

15.0

In [22]:
max_len = np.quantile(sentlen, 0.95)


In [23]:
tok = Tokenizer(char_level=False, split=" ")

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'http',
 2: 'fire',
 3: 'like',
 4: 'amp',
 5: 'new',
 6: 'get',
 7: 'via',
 8: 'people',
 9: 'u',
 10: 'one',
 11: 'emergency',
 12: 'time',
 13: 'video',
 14: 'building',
 15: 'news',
 16: 'police',
 17: 'day',
 18: 'would',
 19: 'body',
 20: 'say',
 21: 'look',
 22: 'back',
 23: 'attack',
 24: 'still',
 25: 'flood',
 26: 'family',
 27: 'disaster',
 28: 'storm',
 29: 'year',
 30: 'bomb',
 31: 'go',
 32: 'need',
 33: 'man',
 34: 'life',
 35: 'see',
 36: 'know',
 37: 'world',
 38: 'got',
 39: 'burning',
 40: 'ca',
 41: 'first',
 42: 'crash',
 43: 'accident',
 44: 'take',
 45: 'suicide',
 46: 'two',
 47: 'service',
 48: 'make',
 49: 'car',
 50: 'woman',
 51: 'home',
 52: 'going',
 53: 'full',
 54: 'weapon',
 55: 'today',
 56: 'nuclear',
 57: 'rt',
 58: 'plan',
 59: 'dead',
 60: 'train',
 61: 'killed',
 62: 'california',
 63: 'think',
 64: 'way',
 65: 'last',
 66: 'may',
 67: 'best',
 68: 'work',
 69: 'watch',
 70: 'war',
 71: 'youtube',
 72: 'many',
 73: 'hiroshima',
 74: 'death',
 

In [24]:
vocab_len = len(tok.index_word)
vocab_len


8765

In [26]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[227, 652, 482, 205, 448, 1039, 726, 1],
 [594, 3489, 56, 30, 653, 3490, 1408, 73, 38, 654, 1409, 248, 168, 1],
 [1721, 1722, 26, 483, 123, 26, 320, 134, 206, 3491, 1],
 [727, 809, 168, 1723, 1410],
 [99, 274, 135, 402, 1202, 403, 1040, 1],
 [3492, 2308, 2309, 1041, 50, 1724, 1203, 2310, 918, 2, 728, 1, 1411],
 [2311, 14, 136, 94, 150, 24, 2312, 2],
 [3493, 364, 63, 321, 1],
 [3494, 3, 3495, 810, 151, 1042, 1],
 [77, 2313, 78, 1725, 275, 365, 1725, 1],
 [5, 811, 105, 811, 53, 100, 3496, 1],
 [5, 811, 105, 811, 3497, 228, 1, 1],
 [35, 595, 366, 367],
 [5, 919, 537, 1, 1412],
 [655, 124, 1413, 1204, 538, 1726, 3498, 150, 124, 2314, 152],
 [920, 322, 404, 1205, 812, 729, 1206, 449, 1],
 [3499, 3500, 921, 188, 169, 189, 596, 1207, 3501, 85, 6, 2315, 101, 539],
 [249, 1727, 3502, 1043, 3503, 3504, 656, 3505],
 [3506, 1208, 484, 137, 190, 36],
 [3507, 1044, 813, 250, 170, 251, 405, 1204],
 [3508, 112, 3, 43, 2316, 540, 368, 169, 596],
 [40, 485, 2317, 36, 1728],
 [814,
  94,
  252,
  5,
  1

In [27]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[   0,    0,    0, ..., 1039,  726,    1],
       [   0,  594, 3489, ...,  248,  168,    1],
       [   0,    0,    0, ...,  206, 3491,    1],
       ...,
       [   0,    0,    0, ..., 2382, 2383,    1],
       [   0,    0,    0, ...,  381, 8759, 2258],
       [   0,    0,    0, ...,  800, 8765,    1]], dtype=int32)

In [28]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [29]:
vocab_len


8765

In [30]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,190, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=50, epochs=25)

ypred = rnn.predict(seqmattest)

#set threshold
ypred = np.where(ypred<0.5,0,1)


Epoch 1/25
72/72 [==============================] - 3s 18ms/step - loss: 0.6157
Epoch 2/25
72/72 [==============================] - 1s 18ms/step - loss: 0.2488
Epoch 3/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0857
Epoch 4/25
72/72 [==============================] - 2s 23ms/step - loss: 0.0526
Epoch 5/25
72/72 [==============================] - 2s 29ms/step - loss: 0.0328
Epoch 6/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0270
Epoch 7/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0248
Epoch 8/25
72/72 [==============================] - 1s 17ms/step - loss: 0.0217
Epoch 9/25
72/72 [==============================] - 1s 17ms/step - loss: 0.0224
Epoch 10/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0203
Epoch 11/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0204
Epoch 12/25
72/72 [==============================] - 1s 18ms/step - loss: 0.0192
Epoch 13/25
72/72 [==================

In [31]:
from sklearn.metrics import classification_report
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76       899
           1       0.66      0.67      0.66       625

    accuracy                           0.72      1524
   macro avg       0.71      0.71      0.71      1524
weighted avg       0.72      0.72      0.72      1524

